In [39]:
# from Linkages import single_linked
from ast import literal_eval as make_tuple
import numpy as np
import pandas as pd
from sklearn.metrics import pairwise_distances
from UltrametricMatrix import count_clusters
from SomeFunc import nprint
from UltrametricMatrix import flatten

In [5]:
points = np.array([
    [1.47464121, 1.61404691],
    [1.02330034, 0.90627011],
    [0.87089836, 1.29429528],
    [1.88869619, 2.06086684]
])

metric='euclidean'
method=single_linked
logs_turn_on=True

In [6]:
dist = pairwise_distances(points, metric=metric)
for i in range(len(dist)):
    dist[i][i] = 0

init_dist = dist.copy()


nprint('Distance matrix: 0 step', logs_turn_on)
nprint(pd.DataFrame(dist), logs_turn_on)

dist[dist == 0] = np.max(dist) + 1


clusters = [str(i) for i in range(len(dist))]


dtype = '<U' + str(5 * sum([len(i) for i in clusters]))
clusters = np.array(clusters, dtype=dtype)

nprint(f"\nClusters:{clusters}\n\n", logs_turn_on)

ultra_dists = []
for k in range(len(dist) - 1):
    indices = np.unravel_index(np.argmin(dist), dist.shape)

    ultra_dists.append(dist[indices])
    
    
    c1 = clusters[indices[0]]
    c2 = clusters[indices[1]]
    new_cluster = f'({c1}, {c2})'
    clusters = np.delete(clusters, indices)
    clusters = np.insert(clusters, 0, new_cluster, axis=0)
    nprint(('Clusters:', clusters), logs_turn_on)

    
    new_dist = np.delete(dist, indices[0], axis=0)
    new_dist = np.delete(new_dist, indices[1] - 1, axis=0)
    new_dist = np.delete(new_dist, indices[0], axis=1)
    new_dist = np.delete(new_dist, indices[1] - 1, axis=1)
    new_dist = np.insert(new_dist, 0, 0, axis=1)
    new_dist = np.insert(new_dist, 0, 0, axis=0)


    ###NEW
    # new_dist = pd.DataFrame(new_dist, columns=clusters, index=clusters)

    # for p, other_cluster in enumerate(new_dist.columns):
    #     options = []
    #     if p != 0:
    #         if type(other_cluster) == str:
    #             other_cluster = flatten(make_tuple(other_cluster))
    #             for j in other_cluster:
    #                 for i in flatten(make_tuple(new_dist.columns[0])):
    #                     options.append(init_dist[i][j])
    #         else:
    #             for i in make_tuple(new_dist.columns[0]):
    #                 options.append(init_dist[i][other_cluster])
    #         new_dist.iloc[0, p] = new_dist.iloc[p, 0] = (np.min(options) + np.max(options)) / 2

    # new_dist = np.array(new_dist)
    

    # for i in range(len(new_dist)):
    #     new_dist[i][i] = 0
    

    # nprint(f'Distance matrix: {k + 1} step', logs_turn_on)
    # nprint(pd.DataFrame(new_dist, columns=clusters, index=clusters), logs_turn_on)
    # nprint("\n\n", logs_turn_on)

    # new_dist[new_dist == 0] = np.max(new_dist) + 1

    # dist = new_dist


    ###OLD
    cur_index = 0
    for i in range(len(dist)):
        if i not in indices:
            cur_index += 1
            new_dist[0][cur_index] = new_dist[cur_index][0] = method(
                dist,
                indices[0],
                indices[1], 
                i,
                clusters_param=count_clusters(make_tuple(new_cluster))
            )

    new_dist[0][0] = np.max(new_dist) + 1

    dist_for_log = new_dist.copy()
    for i in range(len(dist_for_log)):
        dist_for_log[i][i] = 0
    nprint(f'Distance matrix: {k + 1} step', logs_turn_on)
    nprint(pd.DataFrame(dist_for_log), logs_turn_on)
    nprint("\n\n", logs_turn_on)

    dist = new_dist

    ###

Distance matrix: 0 step
          0         1         2         3
0  0.000000  0.839438  0.683189  0.609171
1  0.839438  0.000000  0.416881  1.442915
2  0.683189  0.416881  0.000000  1.274184
3  0.609171  1.442915  1.274184  0.000000

Clusters:['0' '1' '2' '3']


('Clusters:', array(['(1, 2)', '0', '3'], dtype='<U20'))
Distance matrix: 1 step
          0         1         2
0  0.000000  0.683189  1.274184
1  0.683189  0.000000  0.609171
2  1.274184  0.609171  0.000000



('Clusters:', array(['(0, 3)', '(1, 2)'], dtype='<U20'))
Distance matrix: 2 step
          0         1
0  0.000000  0.683189
1  0.683189  0.000000



('Clusters:', array(['((0, 3), (1, 2))'], dtype='<U20'))
Distance matrix: 3 step
     0
0  0.0





# Модифицирование

## модификация функции типов связей

In [40]:
def LanceWillliams(dist, i, j, k, alpha_u, alpha_v, beta, gamma):
    return alpha_u * dist[i][k] + alpha_v * dist[j][k] + \
    beta * dist[i][j] + gamma * abs(dist[i][k] - dist[j][k])

def single_linked(dist, i, j, k, clusters_param=0):
    alpha_u, alpha_v, beta, gamma = 0.5, 0.5, 0, -0.5
    return LanceWillliams(dist, i, j, k, alpha_u, alpha_v, beta, gamma)

In [41]:
dist


array([[2.44291496, 0.83943825, 0.68318851, 0.60917122],
       [0.83943825, 2.44291496, 0.41688115, 1.44291496],
       [0.68318851, 0.41688115, 2.44291496, 1.27418381],
       [0.60917122, 1.44291496, 1.27418381, 2.44291496]])

## модификация функции подсчета

In [43]:
dist = pairwise_distances(points, metric=metric)
for i in range(len(dist)):
    dist[i][i] = 0

init_dist = dist.copy()


nprint('Distance matrix: 0 step', logs_turn_on)

#
dist_dataframe = pd.DataFrame(dist)
nprint(dist_dataframe, logs_turn_on)
# 

dist[dist == 0] = np.max(dist) + 1


clusters = [str(i) for i in range(len(dist))]


dtype = '<U' + str(5 * sum([len(i) for i in clusters]))
clusters = np.array(clusters, dtype=dtype)

nprint(f"\nClusters:{clusters}\n\n", logs_turn_on)

ultra_dists = []

k = 0 # for 
indices = np.unravel_index(np.argmin(dist), dist.shape)

ultra_dists.append(dist[indices])


c1 = clusters[indices[0]]
c2 = clusters[indices[1]]
new_cluster = f'({c1}, {c2})'
clusters = np.delete(clusters, indices)
clusters = np.insert(clusters, 0, new_cluster, axis=0)
nprint(('Clusters:', clusters), logs_turn_on)


new_dist = np.delete(dist, indices[0], axis=0)
new_dist = np.delete(new_dist, indices[1] - 1, axis=0)
new_dist = np.delete(new_dist, indices[0], axis=1)
new_dist = np.delete(new_dist, indices[1] - 1, axis=1)
new_dist = np.insert(new_dist, 0, 0, axis=1)
new_dist = np.insert(new_dist, 0, 0, axis=0)


###NEW
new_dist = pd.DataFrame(new_dist, columns=clusters, index=clusters)
print(f"{new_dist}\n")

for p, other_cluster in enumerate(new_dist.columns):
    print(p, other_cluster)

    options = []
    if p != 0:
        cur_cluster = make_tuple(new_dist.index[0])
        print(type(cur_cluster[0]), type(cur_cluster[1]), type(other_cluster))
        new_dist.iloc[0, p] = new_dist.iloc[p, 0] = method(
            dist,
            cur_cluster[0],
            cur_cluster[1],
            other_cluster,
            clusters_param=count_clusters(make_tuple(new_cluster))
        )




new_dist = np.array(new_dist)


for i in range(len(new_dist)):
    new_dist[i][i] = 0


nprint(f'Distance matrix: {k + 1} step', logs_turn_on)
nprint(pd.DataFrame(new_dist, columns=clusters, index=clusters), logs_turn_on)
nprint("\n\n", logs_turn_on)

new_dist[new_dist == 0] = np.max(new_dist) + 1

dist = new_dist

Distance matrix: 0 step
          0         1         2         3
0  0.000000  0.839438  0.683189  0.609171
1  0.839438  0.000000  0.416881  1.442915
2  0.683189  0.416881  0.000000  1.274184
3  0.609171  1.442915  1.274184  0.000000

Clusters:['0' '1' '2' '3']


('Clusters:', array(['(1, 2)', '0', '3'], dtype='<U20'))
        (1, 2)         0         3
(1, 2)     0.0  0.000000  0.000000
0          0.0  2.442915  0.609171
3          0.0  0.609171  2.442915

0 (1, 2)
1 0
<class 'int'> <class 'int'> <class 'str'>


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [47]:
pd.DataFrame(init_dist)

,0,1,2,3
0,0.000000,0.839438,0.683189,0.609171
1,0.839438,0.000000,0.416881,1.442915
2,0.683189,0.416881,0.000000,1.274184
3,0.609171,1.442915,1.274184,0.000000


In [ ]:
###NEW
for p, other_cluster in enumerate(new_dist.columns):
    options = []
    if p != 0:
        if type(other_cluster) == str:
            other_cluster = flatten(make_tuple(other_cluster))
            for j in other_cluster:
                for i in flatten(make_tuple(new_dist.columns[0])):
                    options.append(init_dist[i][j])
        else:
            for i in make_tuple(new_dist.columns[0]):
                options.append(init_dist[i][other_cluster])
        new_dist.iloc[0, p] = new_dist.iloc[p, 0] = (np.min(options) + np.max(options)) / 2

new_dist = np.array(new_dist)


for i in range(len(new_dist)):
    new_dist[i][i] = 0


nprint(f'Distance matrix: {k + 1} step', logs_turn_on)
nprint(pd.DataFrame(new_dist, columns=clusters, index=clusters), logs_turn_on)
nprint("\n\n", logs_turn_on)

new_dist[new_dist == 0] = np.max(new_dist) + 1

dist = new_dist


###OLD
cur_index = 0
for i in range(len(dist)):
    if i not in indices:
        cur_index += 1
        new_dist[0][cur_index] = new_dist[cur_index][0] = method(
            dist,
            indices[0],
            indices[1], 
            i,
            clusters_param=count_clusters(make_tuple(new_cluster))
        )

new_dist[0][0] = np.max(new_dist) + 1

dist_for_log = new_dist.copy()
for i in range(len(dist_for_log)):
    dist_for_log[i][i] = 0
nprint(f'Distance matrix: {k + 1} step', logs_turn_on)
nprint(pd.DataFrame(dist_for_log), logs_turn_on)
nprint("\n\n", logs_turn_on)

dist = new_dist

###